# 실전 문제 해결 (부족한 데이터셋)

In [1]:
import tensorflow as tf
import numpy as np
import random
from imblearn.over_sampling import BorderlineSMOTE

## 하이퍼 파라미터

In [2]:
EPOCHS = 20

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [4]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)
        
x_test_small = list()
y_test_small = list()

for x, y in zip(x_test, y_test):
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis = 0)
y_test = np.stack(y_test_small, axis = 0)
        
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (불균형한 데이터셋)

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

171/171 [==============================] - 4s 23ms/step - loss: 0.3491 - accuracy: 0.9135 - precision: 0.9286 - recall: 0.9808 - val_loss: 1.3218 - val_accuracy: 0.5345 - val_precision: 0.5180 - val_recall: 0.9940
Epoch 2/20
171/171 [==============================] - 4s 22ms/step - loss: 0.2551 - accuracy: 0.9206 - precision: 0.9342 - recall: 0.9824 - val_loss: 0.4514 - val_accuracy: 0.8045 - val_precision: 0.7908 - val_recall: 0.8280
Epoch 3/20
171/171 [==============================] - 4s 2

## 데이터셋 준비 (BorderlineSMOTE)

In [6]:
# TODO: BorderlineSMOTE 적용하기
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)
smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)


train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (BorderlineSMOTE)

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/20
313/313 [==============================] - 7s 21ms/step - loss: 0.6646 - accuracy: 0.7219 - precision: 0.7257 - recall: 0.7134 - val_loss: 0.5602 - val_accuracy: 0.7430 - val_precision: 0.8958 - val_recall: 0.5500
Epoch 2/20
313/313 [==============================] - 6s 20ms/step - loss: 0.4229 - accuracy: 0.8136 - precision: 0.8301 - recall: 0.7886 - val_loss: 0.4756 - val_accuracy: 0.7820 - val_precision: 0.7402 - val_recall: 0.8690
Epoch 3/20
313/313 [==============================] - 7s 23ms/step - loss: 0.3439 - accuracy: 0.8580 - precision: 0.8828 - recall: 0.8256 - val_loss: 0.4468 - val_accuracy: 0.8020 - val_precision: 0.7687 - val_recall: 0.8640
Epoch 4/20
313/313 [==============================] - 6s 21ms/step - loss: 0.2819 - accuracy: 0.8920 - precision: 0.9188 - recall: 0.8600 - val_loss: 0.4610 - val_accuracy: 0.8095 - val_precision: 0.7657 - val_recall: 0.8920
Epoch 5/20
313/313 [==============================] - 7s 21ms/step - loss: 0.2580 - accuracy: 0.8997